In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import chess
import chess.svg
import cairosvg
from io import BytesIO

# Храним шахматные доски по chat_id
games = {}

def handle_move(update: Update, context: CallbackContext):
    chat_id = update.effective_chat.id
    text = update.message.text.strip()

    # Если партии нет — создаём новую
    if chat_id not in games:
        games[chat_id] = chess.Board()

    board = games[chat_id]

    if board.is_game_over():
        result = board.result()
        update.message.reply_text(f"♟ Игра окончена. Результат: {result}. Напишите /new для новой партии.")
        return

    # Приводим текст к правильному виду (фигуры с заглавной буквы)
    figures = ['k', 'q', 'r', 'b', 'n']
    if text and text[0].lower() in figures:
        san = text[0].upper() + text[1:]
    else:
        san = text.lower()  # пешки и рокировка

    try:
        move = board.parse_san(san)
        board.push(move)
    except ValueError:
        update.message.reply_text("❌ Неверный ход. Пример: e4, nf3, Bb5, Qxd5, O-O")
        return

    # Определяем, чей следующий ход
    side = "♙ Белые" if board.turn == chess.WHITE else "♟ Чёрные"

    # Генерируем изображение доски
    svg_data = chess.svg.board(board=board)
    png_data = cairosvg.svg2png(bytestring=svg_data)

    bio = BytesIO(png_data)
    bio.name = "board.png"
    bio.seek(0)

    # Отправляем картинку доски с подписью
    context.bot.send_photo(chat_id=chat_id, photo=bio, caption=f"Следующий ход: {side}")

def new_game(update: Update, context: CallbackContext):
    chat_id = update.effective_chat.id
    games[chat_id] = chess.Board()
    board = games[chat_id]

    # Генерируем изображение начальной доски
    svg_data = chess.svg.board(board=board)
    png_data = cairosvg.svg2png(bytestring=svg_data)

    bio = BytesIO(png_data)
    bio.name = "board.png"
    bio.seek(0)

    # Отправляем изображение доски с сообщением
    context.bot.send_photo(chat_id=chat_id, photo=bio, caption="♟ Новая партия начата. Ходят белые.")

def main():
    TOKEN = ''
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler('new', new_game))  # обработка команды /new
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_move))  # обработка ходов

    updater.start_polling()
    print("✅ Бот запущен.")
    updater.idle()

if __name__ == '__main__':
    main()


InvalidToken: Invalid token